# Data Wrangling - Optional append

### Use this workbook to download new input data for the finished model or append more historical data to the model training set

In [5]:
import pandas as pd
import yfinance as yf
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from matplotlib import pyplot as plt
import seaborn as sns
import os
import StockMetrics
from statistics import mean
import numpy as np

In [6]:
#import Historical Pricing Data compiled from mass yahoo download
os.chdir(r'C:\Users\nmur1\Google Drive\Springboard\Capstone2\CleanData')
dfHistorical = pd.read_pickle('Daily Pricing Detail.csv').dropna()
print('Last Day: ', dfHistorical.Date.max())

Last Day:  2020-08-21 00:00:00


In [7]:
#import S&P stocks to download
importpath = r'C:\Users\nmur1\Google Drive\Springboard\Capstone2\Stock Import Lists'
importfile = 'SandP.csv'
exportpath = r'C:\Users\nmur1\Google Drive\Springboard\Capstone2\CleanData'
exportfile = 'Momentum'
os.chdir(importpath)

stocks = pd.read_csv(importfile, encoding= 'unicode_escape')


In [10]:
#define date for new stock price download

edate = '2020-08-28'
sdate = '2020-08-22'


from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.4f}'.format

In [11]:

#download new stock data
df = yf.download(list(stocks.Symbol), start = sdate, end = edate, progress = True ).reset_index()
# Melt and Pivot for proper formatting
df = df.melt(id_vars = 'Date', var_name = ['Type', 'Ticker']).set_index(['Date','Ticker']).pivot(columns = 'Type').reset_index()
df.columns = df.columns.droplevel(0)
columns = list(df.columns)
columns[0] = 'Date'
columns[1] = 'Ticker'
df.columns = columns
df = df.sort_values(by = ['Ticker', 'Date'])
df = df.dropna()

    
#append to historical list
df = df[dfHistorical.columns]
dfNew = pd.concat([dfHistorical, df])
dfNew.Date = pd.to_datetime(dfNew.Date)
dfNew = dfNew.sort_values(by = ['Ticker', 'Date'])
    


[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted


In [15]:
dfNew.duplicated().sum()

0

In [16]:
os.chdir(exportpath)
dfNew.to_pickle('Daily Pricing Detail')

In [35]:
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.4f}'.format

#calculate new metrics
#need a solid year of data to properly calculate ADX, DI_Plus, DI_Neg

tday = datetime.today()
delta = tday - timedelta(200)

datefilter = dfNew.Date > delta
Quant = StockMetrics.Momentum(dfNew[datefilter]).reset_index()


In [36]:
NewQuant = Quant.reset_index()


In [37]:
importpath = r'C:\Users\nmur1\Google Drive\Springboard\Capstone2\CleanData'
os.chdir(importpath)

#import datasets from cleaning
fund = 'Fundamental_Final.csv'
analyst = 'Analysts.csv'


dfFund = pd.read_csv(fund).drop(columns = ['Unnamed: 0'])
dfanalyst = pd.read_csv(analyst).drop(columns = ['Unnamed: 0'])


earnings_cal = pd.read_csv('Earnings_Calendar.csv')

#quand = StockMetrics.Momentum(dfpricing).reset_index()
f = StockMetrics.fundamental(dfFund, dfanalyst, NewQuant, earnings_cal)
ftr = StockMetrics.future(f)
ftr.columns

Index(['index', 'Date', 'Close', 'DI_Plus', 'DI_Neg', 'ADX', 'CMF',
       'DI_Plus_Slope', 'DI_Plus_R', 'SMA', 'UpperB', 'LowerB', 'Off_SMA',
       'Off_LB', 'Ticker', 'Year', 'Key', 'eps', 'ROE', 'Sector', 'D2C',
       'epsgrowth', 'Sales', 'Shares', 'Percent_Buy', 'datekey', 'E',
       'PE_Ratio', 'AverageSectorPE', 'Relative_PE', 'Sales_Ratio',
       'AverageSectorSR', 'Relative_SR', 'month', 'E_Season', 'Five_Day',
       'Thirty_Day', 'Sixty_Day', 'Five_Day_avg', 'Thirty_Day_avg',
       'Sixty_Day_avg'],
      dtype='object')

In [38]:
#import datasets from cleaning
fund = 'Fundamental_Final.csv'
analyst = 'Analysts.csv'


dfFund = pd.read_csv(fund).drop(columns = ['Unnamed: 0'])
dfanalyst = pd.read_csv(analyst).drop(columns = ['Unnamed: 0'])


earnings_cal = pd.read_csv('Earnings_Calendar.csv')

#quand = StockMetrics.Momentum(dfpricing).reset_index()
f = StockMetrics.fundamental(dfFund, dfanalyst, NewQuant, earnings_cal)
ftr = StockMetrics.future(f)
ftr.columns

Index(['index', 'Date', 'Close', 'DI_Plus', 'DI_Neg', 'ADX', 'CMF',
       'DI_Plus_Slope', 'DI_Plus_R', 'SMA', 'UpperB', 'LowerB', 'Off_SMA',
       'Off_LB', 'Ticker', 'Year', 'Key', 'eps', 'ROE', 'Sector', 'D2C',
       'epsgrowth', 'Sales', 'Shares', 'Percent_Buy', 'datekey', 'E',
       'PE_Ratio', 'AverageSectorPE', 'Relative_PE', 'Sales_Ratio',
       'AverageSectorSR', 'Relative_SR', 'month', 'E_Season', 'Five_Day',
       'Thirty_Day', 'Sixty_Day', 'Five_Day_avg', 'Thirty_Day_avg',
       'Sixty_Day_avg'],
      dtype='object')

In [39]:
drop = ['Year', 'Key', 'Shares', 'datekey', 'E','month', 'DI_Neg','Five_Day_avg', 'Thirty_Day_avg', 'Sixty_Day_avg']
ftr = ftr.drop(columns = drop)
ftr['Pos_Coef'] = ftr.DI_Plus_Slope * ftr.DI_Plus_R


In [40]:
def EMA(Metric, period):
    
    multiplier = 2/(period+1)
    Base = Metric.reset_index(drop = True)
    Base = Base.rolling(window = period).mean()[period - 1]
    
    Metric = list(Metric)
    period = period -1
    lstlen = len(Metric)
    lstEMA = np.empty(lstlen)

    for i in range(lstlen):

        if i < period:
            lstEMA[i] = 0
        elif i == period:
            lstEMA[i] = Base
        else:
            lstEMA[i] = Metric[i] * multiplier + lstEMA[i-1] * (1-multiplier)


    return lstEMA

In [41]:
ftr['EMA'] = 0
stocks = ftr.Ticker.value_counts().index
for t in stocks:
    close = ftr.loc[ftr['Ticker'] == t,'Close']
    ftr.loc[ftr['Ticker'] == t, 'EMA'] = EMA(close,5)
    
    

In [42]:
ftr['SignalVar'] = ftr.EMA / ftr.SMA


In [43]:
ftr.to_csv('08_29_Model Input.csv')

In [45]:
pd.read_pickle('Historical_Quant')

,Date,Close,DI_Plus,ADX,CMF,DI_Plus_Slope,DI_Plus_R,SMA,UpperB,LowerB,...,E_Season,Five_Day,Thirty_Day,Sixty_Day,Pos_Coef,EMA,SignalVar,Five_Day_avg,Thirty_Day_avg,Sixty_Day_avg
0,2016-01-04,1555.6600,11.7671,16.5890,0.1961,0.0073,0.0008,1657.0880,1718.6384,1595.5376,...,0.0000,1,1,1,0.0000,0.0000,0.0000,1563.6520,1569.4830,1618.3505
1,2016-01-05,1594.0900,10.7071,18.8012,0.2160,-0.0670,0.0407,1651.5050,1714.6233,1588.3867,...,0.0000,0,0,1,-0.0027,0.0000,0.0000,1560.1980,1568.9933,1620.9372
2,2016-01-06,1587.0601,10.7706,20.6828,0.1550,-0.1606,0.1728,1644.8580,1705.6816,1584.0344,...,0.0000,0,0,1,-0.0278,0.0000,0.0000,1548.1340,1569.1453,1623.1122
3,2016-01-07,1552.3199,9.6687,23.1023,0.1474,-0.2431,0.2854,1637.9495,1707.2662,1568.6328,...,0.0000,0,1,1,-0.0694,0.0000,0.0000,1540.6700,1569.9673,1625.7172
4,2016-01-08,1541.4200,8.9108,25.4337,0.1618,-0.3542,0.4429,1633.3710,1714.4858,1552.2562,...,0.0000,0,1,1,-0.1569,1566.1100,0.9588,1534.7900,1571.4527,1628.7195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568835,2020-08-10,26.3100,24.0661,34.5182,0.1798,0.0816,0.0865,25.3905,26.8882,23.8928,...,0.0000,0,0,0,0.0071,26.2610,1.0343,0.0000,0.0000,0.0000
568836,2020-08-11,26.5100,28.1042,36.0551,0.1017,0.1429,0.1900,25.5035,26.9808,24.0262,...,0.0000,0,0,0,0.0271,26.3440,1.0330,0.0000,0.0000,0.0000
568837,2020-08-12,26.2400,25.5900,36.9583,0.0567,0.1515,0.2137,25.5680,27.0556,24.0804,...,0.0000,0,0,0,0.0324,26.3094,1.0290,0.0000,0.0000,0.0000
568838,2020-08-13,26.6900,23.4143,37.2188,0.0445,0.0856,0.0586,25.6275,27.1926,24.0624,...,0.0000,0,0,0,0.0050,26.4362,1.0316,0.0000,0.0000,0.0000
